In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.cuda.amp import autocast, GradScaler

import torch.distributed as dist

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.tensorboard import SummaryWriter

# create TensorBoard summary writer
log_dir = '/workspace/w251-hw9/logs'
writer = SummaryWriter(log_dir)

2023-03-13 02:05:13.012162: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 02:05:13.860198: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/usr/local/lib/python3.8/dist-packages/torch/lib:/usr/local/lib/python3.8/dist-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda-11/lib64
2023-03-13 02:05:13.860284: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dl

In [3]:
GPU=0

In [4]:
SEED=1

random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

scaler = GradScaler()

In [5]:
torch.cuda.device_count()

1

In [6]:
START_EPOCH = 0

In [7]:
ARCH = 'resnet18'
EPOCHS = 2
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
PRINT_FREQ = 50
TRAIN_BATCH=1024
VAL_BATCH=1024
WORKERS=1
RANK=0

TRAINDIR="/data/train"
VALDIR="/data/val"

### Check if cuda is available here

In [8]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!
if not torch.cuda.is_available():
    print("GPU not detected...")

In [9]:
# set your active device to your GPU in this cell
#device = torch.device("cuda")
GPU = torch.device('cuda:0')
cudnn.benchmark = True

### Fill in the heart of the train section below

In [10]:
global_step = 0

In [11]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    ######################
    # switch model to train mode here
    model.train()
    ################
    
    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        if GPU is not None:
            # send the images to cuda device
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            # send the target to cuda device
            target = target.cuda(GPU, non_blocking=True)

        
        with autocast():
            # compute output
            output = model(images)
            # compute loss
            loss = criterion(output, target)


        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))


        #### zero out gradients in the optimier
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [12]:
def validate(val_loader, model, criterion):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output & loss
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)
        
        writer.add_scalar("Loss/val", loss, global_step = global_step)
        writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/val", top5.avg, global_step = global_step)
        global_step = global_step + 1
        
        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [13]:
def save_checkpoint(state, is_best, filename='checkpoints/single_vm_checkpoint.pth.tar'):
    # save the model state!
    torch.save(state, filename)
    
    if is_best:
        shutil.copyfile(filename, 'checkpoints/single_vm_model_best.pth.tar')

In [14]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [15]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [16]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [17]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [18]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.24205776, 0.23828046, 0.25874835]

In [19]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [20]:
IMG_SIZE = 224
INPUT_SIZE = 256
NUM_CLASSES = 1000

### Initialize the model using the architecture you selected above

In [21]:
# select the model
model = models.__dict__[ARCH](pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
inf = model.fc.in_features

In [23]:
model.fc = nn.Linear(inf,NUM_CLASSES)

### Send the model to the cuda device

In [24]:
model.cuda(GPU)
next(model.parameters()).is_cuda

True

In [25]:
torch.cuda.memory_allocated()

46861312

### Instantiate the loss to cross entropy

In [26]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss().cuda(GPU)

### Instantiate the optimizer to SGD

In [27]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

#### Create the learning rate scheduler

In [28]:
# use CosineAnnealingLR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=0.00001)

In [29]:
transform_train = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

### Create the train dataset object

In [30]:
train_dataset = datasets.ImageFolder(root=TRAINDIR, 
                                     transform=transform_train)

In [31]:
transform_val = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

### Create the val dataset object

In [32]:
# use torchvision.datasets.CIFAR10
# val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)
val_dataset = datasets.ImageFolder(root=VALDIR, transform=transform_val)

### Create the train dataloader

In [33]:
# fill this in
train_loader = torch.utils.data.DataLoader(train_dataset, 
                     batch_size=TRAIN_BATCH, 
                     shuffle=True, 
                     num_workers=WORKERS,
                     pin_memory=True)

In [34]:
# fill this in..
val_loader = torch.utils.data.DataLoader(val_dataset, 
                            batch_size=VAL_BATCH,
                            shuffle=False, 
                            num_workers=WORKERS,
                            pin_memory=True)

In [35]:
best_acc1 = 0

In [36]:
%%time

for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

    if RANK == 0:
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': ARCH,
            'state_dict': model.state_dict(),
            'best_acc1': best_acc1,
            'optimizer' : optimizer.state_dict(),
        }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
# close TensorBoard writer
writer.close()

Epoch: [0][   0/1252]	Time 10.928 (10.928)	Data  6.082 ( 6.082)	Loss 7.1260e+00 (7.1260e+00)	Acc@1   0.10 (  0.10)	Acc@5   0.68 (  0.68)
Epoch: [0][  50/1252]	Time  5.296 ( 5.329)	Data  4.727 ( 4.677)	Loss 2.5539e+00 (4.4737e+00)	Acc@1  46.68 ( 24.76)	Acc@5  73.73 ( 44.95)
Epoch: [0][ 100/1252]	Time  6.021 ( 5.348)	Data  5.454 ( 4.738)	Loss 1.8583e+00 (3.3156e+00)	Acc@1  57.62 ( 38.60)	Acc@5  84.18 ( 62.20)
Epoch: [0][ 150/1252]	Time  6.778 ( 5.705)	Data  6.210 ( 5.109)	Loss 1.8893e+00 (2.8362e+00)	Acc@1  54.39 ( 44.64)	Acc@5  81.45 ( 68.90)
Epoch: [0][ 200/1252]	Time  6.550 ( 5.951)	Data  5.982 ( 5.362)	Loss 1.7766e+00 (2.5805e+00)	Acc@1  58.59 ( 47.87)	Acc@5  83.01 ( 72.32)
Epoch: [0][ 250/1252]	Time  6.860 ( 6.102)	Data  6.292 ( 5.517)	Loss 1.7409e+00 (2.4217e+00)	Acc@1  57.71 ( 49.87)	Acc@5  83.69 ( 74.46)
Epoch: [0][ 300/1252]	Time  6.759 ( 6.201)	Data  6.192 ( 5.619)	Loss 1.7538e+00 (2.3120e+00)	Acc@1  57.62 ( 51.22)	Acc@5  82.71 ( 75.88)
Epoch: [0][ 350/1252]	Time  6.824 ( 6.284